# Processing the Ranks 

In [400]:
import os
import re
import datetime
from functools import reduce

import numpy as np
import pandas as pd
from scipy.stats import mode

In [393]:
DATA_DIR = '/Users/vineethsuhas/vineeth/handsOn/hackathons/kaggle/GoogleMLCompNCAAM/data/raw/'

In [364]:
def get_dates(sdt, edt):
    dates = [datetime.datetime.date(sdt)]
    while sdt < edt:
        sdt = sdt + datetime.timedelta(days=7)
        dates.append(datetime.datetime.date(sdt))
    return dates

In [338]:
def read_ranks(path):
    df = pd.read_csv(path)

    cols = list(df.columns)
    cols[4] = 'Delta'
    df.columns = cols
    to_remove = [cols[0], 'Conf', 'CMP', 'Sort', cols[-1]]
    df.drop(to_remove, axis=1, inplace=True)
    return df

In [339]:
def process_ranks(df, week_num):
    # Add the more repeating rank which indicates more popular rank
    df['popularity_rank'] = df.apply(lambda x: mode(x[3:-2])[0][0], axis=1)
    
    # Drop various systems rankings, Just include Mean and StDev
    df.drop(df.columns[3:-3], axis=1, inplace=True)
    
    # Converting W-L into wins and losses
    # (# https://www.geeksforgeeks.org/split-a-text-column-into-two-columns-in-pandas-dataframe/)
    df = pd.concat([df, pd.DataFrame(df['W-L'].str.split('-').to_list(), columns=['wins', 'losses'])], axis=1).drop('W-L', axis=1)
    
    # Convert the Delta to integer
    df.Delta = df.Delta.fillna(0).astype('int')
    
    # Add the week number string to column names
    cols = ['Team'] + list(map(lambda x: 'week{0}_'.format(week_num) + x, df.columns[1:]))
    df.columns = cols
    
    return df

In [ ]:
def assert_team_names(scraped_teams, cur_teams):
    scraped_teams = list(ranks['Team'])
    cur_teams = list(teams['TeamName'])
    for each in scraped_teams:
        assert each in cur_teams, "Team `{0}` Does Not Exist".format(each)

In [734]:
OUTPUT_SEASONED_RANKS_PATH = DATA_DIR + 'MRanks/'
if not os.path.exists(OUTPUT_SEASONED_RANKS_PATH):
    os.mkdir(OUTPUT_SEASONED_RANKS_PATH)

def write_season_ranks(sd, ed, season):
    dates = get_dates(sd, ed)

    dfs = []
    for i, date in enumerate(dates):
        path = OUTPUT_PATH + 'ranks_' + str(date).replace('-', '_') + '.csv'
        df = read_ranks(path)
        processed_df = process_ranks(df, i+1)
        dfs.append(processed_df)

    df_merged = reduce(lambda left, right: pd.merge(left, right, on='Team'), dfs)
    
    # Append the TeamID
    try:
        teams = pd.read_csv(DATA_DIR + 'MDataFiles_Stage1/MTeams.csv')
        assert_team_names(df_merged['Team'], teams['TeamName'])
        df_merged = df_merged.merge(teams[['TeamID', 'TeamName']], left_on='Team', right_on='TeamName').drop('TeamName', axis=1)
    except AssertionError as e:
        print("Continuing without TeamID for season {0} because {1}".format(season, str(e)))
        pass
    
    # Write to File
    path = OUTPUT_SEASONED_RANKS_PATH + 'season_{0}'.format(season) + '.csv'
    df_merged.to_csv(path, index=False)

In [739]:
write_season_ranks(datetime.datetime(2014, 11, 17), datetime.datetime(2015, 3, 16), '2015')
write_season_ranks(datetime.datetime(2015, 11, 16), datetime.datetime(2016, 3, 14), '2016')
write_season_ranks(datetime.datetime(2016, 11, 14), datetime.datetime(2017, 3, 13), '2017')
write_season_ranks(datetime.datetime(2017, 11, 13), datetime.datetime(2018, 3, 12), '2018')
write_season_ranks(datetime.datetime(2018, 11, 12), datetime.datetime(2019, 3, 18), '2019')

# Merging Ranks with Events 

In [740]:
def get_processed_ranks_to_merge(season):
    ranks = pd.read_csv(DATA_DIR + 'MRanks/season_{0}.csv'.format(season))
    
    # Pivot and stack to get the weekly attributes into rows
    unp_ranks = ranks.pivot_table(index=['TeamID', 'Team']).stack().reset_index()
    
    # Get the week numbers and add the attributes as a new_col which can be further 
    unp_ranks['week'] = unp_ranks.level_2.apply(lambda x: re.findall(r'\d+', x)[0])
    unp_ranks["new_cols"] = unp_ranks.level_2.str.split('_').apply(lambda x: "_".join(x[1:]))
    
    # Dropping and Renaming the columns 
    unp_ranks.drop(['level_2', 'Team'], axis=1, inplace=True)
    unp_ranks.rename(columns={0: 'values'}, inplace=True)
    
    # Pivot using TeamID and week which will give all the attributes as the seperate columns
    rank_pivots = unp_ranks.pivot_table(index=['TeamID', 'week'], columns=['new_cols'])
    rank_pivots.columns = rank_pivots.columns.droplevel()
    rank_pivots.reset_index(inplace=True)
    
    # Convert the TeamID and week to int.
    rank_pivots['week'] = rank_pivots.week.astype('int')
    rank_pivots['TeamID'] = rank_pivots.TeamID.astype('int')
    
    return rank_pivots

In [741]:
def get_events_for_merging(season):
    events = pd.read_csv(DATA_DIR + 'HistoricalEvents/MEvents{0}.csv'.format(season))
    events = events[['Season', 'DayNum', 'WTeamID', 'LTeamID']]
    events['week_num'] = np.ceil(events['DayNum'] / 7).astype('int')
    return events

In [746]:
def merge_events_and_ranks(events, rank_pivots):
    # First merge and get the winning teams attributes
    events_with_ranks = pd.merge(events, 
                                 rank_pivots, 
                                 left_on=['week_num', 'WTeamID'], 
                                 right_on=['week', 'TeamID'], 
                                 how='left')
    events_with_ranks.rename(columns={'Delta':'WDeltaRank', 
                                      'Mean': 'WMeanRank', 
                                      'StDev': 'WStDevRank', 
                                      'losses': 'WLosses', 
                                      'popularity_rank': 'WPopularityRank', 
                                      'wins': 'WWins'},                       
                             inplace=True)
    events_with_ranks = events_with_ranks.drop(['TeamID', 'week'], axis=1)
    
    # Repeat the process to get the loosing teams attributes
    events_with_ranks = pd.merge(events_with_ranks, 
                                 rank_pivots, 
                                 left_on=['week_num', 'LTeamID'], 
                                 right_on=['week', 'TeamID'], 
                                 how='left')
    events_with_ranks.rename(columns={'Delta':'LDeltaRank', 
                                      'Mean': 'LMeanRank', 
                                      'StDev': 'LStDevRank', 
                                      'losses': 'LLosses', 
                                      'popularity_rank': 'LPopularityRank', 
                                      'wins': 'LWins'},                       
                             inplace=True)
    events_with_ranks = events_with_ranks.drop(['TeamID', 'week', 'week_num'], axis=1)
    
    return events_with_ranks

In [835]:
OUTPUT_EVENT_RANKS_PATH = DATA_DIR + 'MEventsRanked/'
if not os.path.exists(OUTPUT_EVENT_RANKS_PATH):
    os.mkdir(OUTPUT_EVENT_RANKS_PATH)

seasons = ['2015', '2016', '2017', '2018', '2019']

for season in seasons:
    ranks = get_processed_ranks_to_merge(season)
    events = get_events_for_merging(season)
    
    # Fill the ranks for additional weeks in events with the latest rank
    for i in range(0, (events.week_num.max() - ranks.week.max())):
        add_week_data = ranks[ranks.week == ranks.week.max()]
        add_week_data['week'] += 1
        ranks = pd.concat([ranks, add_week_data])
    
    merged_events = merge_events_and_ranks(events, ranks)
    
    # Write to file:
    file_path = OUTPUT_EVENT_RANKS_PATH + 'MEventsRanked{0}.csv'.format(season)
    merged_events.to_csv(file_path, index=False)

/Users/vineethsuhas/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
